In [2]:
#Finance Model

#importing necessary modules 

import classiq
from classiq.builtin_functions import PiecewiseLinearAmplitudeLoading, PhaseEstimation
from classiq import QUInt, Model, synthesize, show, QReg, ControlState, execute,set_constraints
from classiq import FunctionGenerator, FunctionLibrary, RegisterUserInput
from classiq.builtin_functions import LinearPauliRotations, StatePreparation,AmplitudeEstimation, ArithmeticOracle, GroverOperator
from classiq.builtin_functions import XGate, UGate, ZGate
from classiq.model import Constraints, TranspilerBasisGates
from classiq.execution import ExecutionDetails
from classiq.execution import QaeWithQpeEstimationMethod
from classiq.model import Constraints, OptimizationParameter
import numpy as np
import matplotlib.pyplot as plt

In [3]:
qmci_library = FunctionLibrary()
function_generator =  FunctionGenerator(function_name="state_amp_load")

sp_num_qubits = 2

#number of possible option prices is equal to 2^{number of qubits}. Here we consider 4 possible option price paths

input_dict = function_generator.create_inputs({"io": QUInt[sp_num_qubits], "ind": QUInt[1]})
# probabilities= [0.20174669, 0.31564887, 0.29626157, 1- (0.20174669+0.31564887+0.29626157)]
probabilities= [0.20, 0.35, 0.25, 0.20]

#specifying the probabilities of 4 price paths- this has been taken from a log-normal distribution with mean = 0.01 and std_dev = 0.1

sp_params = StatePreparation(
    probabilities=probabilities, error_metric={"KL": {"upper_bound": 0.00}}
)

sp_output = function_generator.StatePreparation(
    params=sp_params, strict_zero_ios=False, in_wires={"IN": input_dict["io"]})
#the in_wires argument specifies the input register to the StatePreparation operation

amplitude_loading_params = PiecewiseLinearAmplitudeLoading(
    num_qubits=2,
    breakpoints=[45, 70, 87.5],
    affine_maps=[{"offset": 0, "slope": 0}, {"offset": -70, "slope": 1}])

al_output = function_generator.PiecewiseLinearAmplitudeLoading(
    params=amplitude_loading_params, strict_zero_ios=False, in_wires={"state":sp_output["OUT"], "target": input_dict["ind"]}
)
function_generator.set_outputs({"io": al_output["state"], "ind": al_output["target"]})

qmci_library.add_function(function_generator.to_function_definition())

In [9]:

state_amp= qmci_library.get_function('state_amp_load')

oracle_params = ArithmeticOracle(
    expression="ind == 1",
    definitions={"ind": RegisterUserInput(size= 1)})

grover_operator_params = GroverOperator(
    oracle_params=oracle_params,
    state_preparation= 'state_amp_load',
    state_preparation_params= state_amp,
)
    
qae_params = AmplitudeEstimation(grover_operator=grover_operator_params, estimation_register_size=3)


ValidationError: 1 validation error for GroverOperator
state_preparation_params -> state_preparation_params
   (type=assertion_error)